In [0]:
import numpy as np
import pandas as pd
import time

### Create synthetic dataset

In [0]:
n_rows = 1000000
n_cols = 500
n_clusters_data = 200
cluster_std = 1.0
# our transform code doesn't yet work with np.float32
#dtype='float32'
dtype='float64'
from cuml.datasets import make_blobs
data, _ = make_blobs(
        n_rows, n_cols, n_clusters_data, cluster_std=cluster_std, random_state=0, dtype=dtype
    )  # make_blobs creates a random dataset of isotropic gaussian blobs.

data = data.get()

In [0]:
pd_data = pd.DataFrame({"features": list(data)})
df = spark.createDataFrame(pd_data)


### Run Cuml-on-Spark KMeans.fit

In [0]:
from sparkcuml.cluster import SparkCumlKMeans

In [0]:
num_workers = 2
n_clusters = 200
max_iter = 30

In [0]:
gpu_kmeans = SparkCumlKMeans(num_workers=num_workers,tol=0.0)\
            .setK(n_clusters)\
            .setFeaturesCol("features")\
            .setMaxIter(max_iter)

KMeans estimator can be persisted and reloaded.

In [0]:
gpu_kmeans.write().overwrite().save("dbfs:/tmp/sparkcuml-kmeans-estimator")
gpu_kmeans_loaded = SparkCumlKMeans.load("dbfs:/tmp/sparkcuml-kmeans-estimator")


In [0]:
start_time = time.time()
gpu_model = gpu_kmeans_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 40.159857511520386 sec

In [0]:
gpu_kmeans_loaded.getOrDefault("n_clusters")

Out[23]: 200

In [0]:
sorted_clusters = sorted(gpu_model.cluster_centers_)

In [0]:
[vec[0:10] for vec in sorted_clusters[0:2]]

Out[25]: [[-9.973976745480062,
 -7.923049703243916,
 4.408621959223942,
 5.516847616416288,
 8.820765124958875,
 9.503997225930416,
 -0.6309299087612391,
 2.4173651733639967,
 -4.011074904174833,
 4.086825561920079],
 [-9.954177187298777,
 -5.060382510260128,
 -7.596014324396889,
 0.6035457653160903,
 -9.186399104683435,
 3.9573064373066047,
 2.3060651828213077,
 -0.255431677280924,
 0.022057060784275277,
 0.01900963311811959]]

### Cuml-on-Spark KMeans model save, load, and transform

In [0]:
gpu_model.write().overwrite().save("dbfs:/tmp/sparkcuml-kmeans-model")

In [0]:
gpu_model_loaded = gpu_model.read().load("dbfs:/tmp/sparkcuml-kmeans-model")

In [0]:
[vec[0:10] for vec in sorted(gpu_model_loaded.cluster_centers_)[0:2]]

Out[40]: [[-9.973976745480062,
 -7.923049703243916,
 4.408621959223942,
 5.516847616416288,
 8.820765124958875,
 9.503997225930416,
 -0.6309299087612391,
 2.4173651733639967,
 -4.011074904174833,
 4.086825561920079],
 [-9.954177187298777,
 -5.060382510260128,
 -7.596014324396889,
 0.6035457653160903,
 -9.186399104683435,
 3.9573064373066047,
 2.3060651828213077,
 -0.255431677280924,
 0.022057060784275277,
 0.01900963311811959]]

In [0]:
transformed_df = gpu_model_loaded.setPredictionCol("transformed").transform(df)

In [0]:
transformed_df.printSchema()

root
-- transformed: integer (nullable = false)

In [0]:
transformed_df.count()

Out[43]: 1000000

In [0]:
transformed_df.show(10)

+-----------+
transformed|
+-----------+
 11|
 94|
 89|
 10|
 106|
 98|
 94|
 6|
 94|
 86|
+-----------+
only showing top 10 rows

### Run PySpark ML KMeans.fit

Timed with 6 tasks per node parallelism, 2 nodes.

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
spark_ml_kmeans = KMeans()\
    .setFeaturesCol("features")\
    .setK(n_clusters)\
    .setTol(0.0)\
    .setMaxIter(max_iter)

Convert array sql type to VectorUDT expected by Spark ML algos

In [0]:
from pyspark.ml.functions import array_to_vector

In [0]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

In [0]:
start_time = time.time()
spark_ml_kmeans_model = spark_ml_kmeans.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 241.63387727737427 sec

In [0]:
type(spark_ml_kmeans_model.clusterCenters()[0])

Out[31]: numpy.ndarray

In [0]:
sorted_spark_ml_cluster_centers = sorted([vec.tolist() for vec in spark_ml_kmeans_model.clusterCenters()])
[vec[0:10] for vec in sorted_spark_ml_cluster_centers]

Out[36]: [[-9.974191443752346,
 -7.923021335805827,
 4.408700303680475,
 5.51656349730256,
 8.82117043945357,
 9.503642763468592,
 -0.6308615253046943,
 2.417414317457042,
 -4.011049318931834,
 4.087072227549893],
 [-9.954177316376787,
 -5.060382328146701,
 -7.596014528355314,
 0.6035457126499217,
 -9.186399194454653,
 3.9573063821473973,
 2.3060651774754515,
 -0.2554316899087624,
 0.022056937905841015,
 0.019009637858647392],
 [-9.914663767336947,
 4.607563731678203,
 -7.479176699042988,
 1.7468020073608983,
 -5.8357681948545945,
 6.038814655325799,
 10.710043158087263,
 -3.363986535205749,
 2.7312080250929247,
 -7.497373661711153],
 [-9.866471733238676,
 9.59391662445215,
 -9.85422955220256,
 -1.4821380175927037,
 9.474989807791136,
 5.017936284775242,
 -4.453566169900044,
 1.354924848572015,
 8.101219635014468,
 5.236360220456646],
 [-9.843678559164283,
 3.4067333710245773,
 -3.6334518751176623,
 -3.792494762966183,
 -1.3011978484900075,
 -5.138451745245338,
 3.8878361623290285,
 -4.723498908085043,
 -7.106244978500376,
 8.194257777704218],
 [-9.715452796272666,
 3.8023918319515806,
 1.5064448896761553,
 -7.626709945193139,
 -1.883488058458599,
 3.177717306279881,
 0.477636731556634,
 9.9104043966488,
 -7.2790644027364415,
 -7.3912394372302845],
 [-9.709203456035254,
 2.2610641206859707,
 6.3507853102022205,
 6.485493139617813,
 7.12130085928903,
 0.6933664879415241,
 -4.9742144661115395,
 -2.1155312514935365,
 -0.7626242642131932,
 9.46593191428],
 [-9.651224007749832,
 3.52218680651136,
 5.732697309771117,
 5.175606968121489,
 -5.721892406182501,
 -6.644460602889856,
 -2.1232860638009243,
 -5.337577628716021,
 -3.8334458004836103,
 -5.6422737906387],
 [-9.642477275580257,
 9.826762883002074,
 -6.505913355260311,
 -2.391505026087839,
 -5.209839643049399,
 7.861144154328028,
 -3.4562849809387983,
 -7.194500540602979,
 -8.479091104137218,
 7.02228588426521],
 [-9.5260001777855,
 3.4983914598567076,
 -2.6147908431937394,
 0.18571722536825458,
 3.614522765048635,
 9.656217491086622,
 -0.9873374208172775,
 7.164175578783082,
 8.155532940911865,
 4.763041780646303],
 [-9.518949138528118,
 -0.6267601692492022,
 -1.0086205576169625,
 -3.006209344746223,
 5.900568298590255,
 7.359087043367324,
 6.017242790350443,
 -4.02208803486319,
 -2.144777215015693,
 3.568852440345234],
 [-9.507802712256012,
 7.604165667820464,
 -6.712351375382649,
 -0.46870060915636547,
 -2.046364743797664,
 2.6438486125035596,
 1.5776690473689239,
 2.0134623396115545,
 3.8705209775633143,
 -8.629571672980655],
 [-9.375550062521706,
 -3.0454264776022986,
 4.622914259960638,
 -2.403895776725338,
 -2.01161752611057,
 -9.740402521848534,
 3.5213634933158016,
 -5.386898964985501,
 -7.877706599792257,
 -7.561252999839132],
 [-9.366099727350113,
 2.8485398471600285,
 -5.858576803337404,
 2.0223790075582313,
 -6.3686285115117895,
 4.701786552502296,
 9.947969035055857,
 -3.064290568272292,
 3.0372355248105993,
 -8.556798438640403],
 [-9.339164056833363,
 -3.7956101024857483,
 -3.2379014060514226,
 6.862168277718081,
 -4.912105189235332,
 -2.627599870764234,
 2.1652364381940794,
 -7.429121451039419,
 -4.8529382457216705,
 -4.9029642796706465],
 [-9.151423682564607,
 6.3142831317885175,
 -7.7925669384578296,
 -8.963609687762801,
 6.578488440200722,
 5.984985292152767,
 -2.560886661499073,
 -2.225179698179845,
 -4.803019089400736,
 -3.6614223797073078],
 [-8.923562602679322,
 -8.061386103887688,
 4.02563891340278,
 6.905260568536857,
 6.839083408707923,
 11.237324941079326,
 -0.9658281698117809,
 2.175646312842241,
 -4.136457765888667,
 2.8798688835125126],
 [-8.900416329396014,
 0.4876215513274028,
 -5.3926212271252005,
 3.4238483775629276,
 -4.836885534575124,
 -4.461983840544338,
 -7.9611318660968236,
 -2.657318385589158,
 7.704712345738049,
 -5.076570635611068],
 [-8.523198446517377,
 -9.222104367234053,
 7.481377957019773,
 -5.984943350808029,
 0.30847996468374256,
 8.729017732766144,
 -2.180804778809095,
 5.873874740296887,
 -4.895391961534976,
 0.1131717332213440

In [0]:
spark_transformed = spark_ml_kmeans_model.setPredictionCol("transformed").transform(vector_df)

In [0]:
spark_transformed.filter(spark_transformed.transformed >= 0).count()

Out[46]: 1000000

In [0]:
spark_transformed.show(10)

+--------------------+-----------+
 features|transformed|
+--------------------+-----------+
[1.86171810337721...| 105|
[1.46597433706294...| 67|
[3.16600204382223...| 26|
[3.06821529911693...| 25|
[9.04055967294711...| 48|
[1.09100676150392...| 11|
[2.42370259615193...| 67|
[-1.0812963982713...| 59|
[1.96545020698749...| 67|
[0.02137651071046...| 188|
+--------------------+-----------+
only showing top 10 rows

### Running cuml.KMeans.fit single-node on driver

In [0]:
from cuml import KMeans as KMeansCuml

In [0]:
cuml_kmeans = KMeansCuml(n_clusters=200, max_iter=max_iter, tol=0.0, verbose=6)

In [0]:
cuml_model = cuml_kmeans.fit(data)

[D] [22:20:27.003405] /project/cpp/src/kmeans/sg_impl.cuh:623 

KMeans.fit (Iteration-1/1): initialize cluster centers using k-means++ algorithm.
[D] [22:20:27.024590] /project/cpp/src/kmeans/sg_impl.cuh:433 KMeans||: psi = 3.49308e+10, log(psi) = 24.2766, niter = 8 
[D] [22:20:27.024759] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 0: # potential centroids sampled - 1
[D] [22:20:27.039190] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 1: # potential centroids sampled - 384
[D] [22:20:27.114286] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 2: # potential centroids sampled - 755
[D] [22:20:27.265459] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 3: # potential centroids sampled - 1158
[D] [22:20:27.511553] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 4: # potential centroids sampled - 1550
[D] [22:20:27.827013] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 5: # potential centroids sampled - 1928
[D] [22:20:28.214783] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 6: # potential centroids sampled - 2306
[D] [22:20:28.680020] /project/cpp/src/kmeans/sg_impl.cuh:437 KMeans|| - Iteration 7: # potential centroids sampled - 2688
[D] [22:20:29.211149] /project/cpp/src/kmeans/sg_impl.cuh:485 KMeans||: total # potential centroids sampled - 3084
[D] [22:20:29.825682] /project/cpp/src/kmeans/common.cuh:696 Run sequential k-means++ to select 200 centroids from 3084 input samples (8 seeding trials per iterations)
[D] [22:20:29.826235] /project/cpp/src/kmeans/common.cuh:760 k-means++ - Sampled 1/200 centroids
[D] [22:20:29.826633] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 2/200 centroids
[D] [22:20:29.827502] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 3/200 centroids
[D] [22:20:29.827801] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 4/200 centroids
[D] [22:20:29.828237] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 5/200 centroids
[D] [22:20:29.828521] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 6/200 centroids
[D] [22:20:29.828763] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 7/200 centroids
[D] [22:20:29.829000] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 8/200 centroids
[D] [22:20:29.829351] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 9/200 centroids
[D] [22:20:29.829656] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 10/200 centroids
[D] [22:20:29.829975] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 11/200 centroids
[D] [22:20:29.830276] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 12/200 centroids
[D] [22:20:29.830642] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 13/200 centroids
[D] [22:20:29.831020] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 14/200 centroids
[D] [22:20:29.831278] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 15/200 centroids
[D] [22:20:29.831507] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 16/200 centroids
[D] [22:20:29.831735] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 17/200 centroids
[D] [22:20:29.831968] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 18/200 centroids
[D] [22:20:29.832194] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 19/200 centroids
[D] [22:20:29.832425] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 20/200 centroids
[D] [22:20:29.832644] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 21/200 centroids
[D] [22:20:29.832867] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 22/200 centroids
[D] [22:20:29.833091] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 23/200 centroids
[D] [22:20:29.833313] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 24/200 centroids
[D] [22:20:29.833536] /project/cpp/src/kmeans/common.cuh:855 k-means++ - Sampled 25/200 centroids
[D] [22:20:29.833761] /projec

In [0]:
cuml_model.n_iter_

Out[8]: 31

In [0]:
cuml_centers_sorted = sorted([vec.tolist() for vec in cuml_model.cluster_centers_])
[vec[0:10] for vec in cuml_centers_sorted[0:2]]

Out[11]: [[-9.973976635156735,
 -7.9230496254742055,
 4.408621984034132,
 5.516847426370484,
 8.820765187556056,
 9.503997227213757,
 -0.6309300116356099,
 2.417364886255936,
 -4.011074959587518,
 4.086825406175116],
 [-9.954177316376782,
 -5.060382328146701,
 -7.596014528355314,
 0.6035457126499217,
 -9.186399194454655,
 3.9573063821473973,
 2.30606517747545,
 -0.2554316899087624,
 0.022056937905841026,
 0.0190096378586474]]